#  残差网络 (ResNets)

In [9]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

#batch_size:表示单次传递给程序用以训练的参数个数
batch_size = 64

'''
transforms.ToTensor()为：将0-255的像素转换为0-1的值
transforms.Normalize((0.1307,), (0.3081,))为：归一化操作
其中由于MNIST数据集为单通道图像，那么分别只有一个均值和一个标准差（一般的图像会有三个通道，代表三原色）
'''
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])

train_dataset = datasets.MNIST(root='../dataset/mnist/',train=True,download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True, batch_size = batch_size)
test_dataset = datasets.MNIST(root = '../dataset/mnist/',train=False,download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=True,batch_size = batch_size)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock,self).__init__()
        self.channels = channels
        # kernel_size()卷积核的大小
        
        self.conv1 = nn.Conv2d(channels,channels,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(channels,channels,kernel_size=3,padding=1)
            
    def forward(self,x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x + y)
    
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size=5)
        self.conv2 = nn.Conv2d(16,32,kernel_size=5)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(512,10)
        
    def forward(self,x):
        in_size = x.size(0)
        
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        
        x = x.view(in_size,-1)
        x = self.fc(x)
        
        return x
    
model = Net()

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.5)

def train(epoch):
    running_loss = 0
    for batch_idx, data in enumerate(train_loader,0):
        inputs, target = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss:%.3f'% (epoch+1,batch_idx+1,running_loss/300))
            running_loss == 0.0
            
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _,predicted = torch.max(outputs.data,dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set:%d %%' % (100*correct/total))
    
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()
        

[1,   300] loss:0.531
[1,   600] loss:0.680
[1,   900] loss:0.790
accuracy on test set:97 %
[2,   300] loss:0.092
[2,   600] loss:0.168
[2,   900] loss:0.237
accuracy on test set:98 %
[3,   300] loss:0.059
[3,   600] loss:0.118
[3,   900] loss:0.175
accuracy on test set:98 %
[4,   300] loss:0.047
[4,   600] loss:0.095
[4,   900] loss:0.143
accuracy on test set:98 %
[5,   300] loss:0.041
[5,   600] loss:0.080
[5,   900] loss:0.120
accuracy on test set:98 %
[6,   300] loss:0.034
[6,   600] loss:0.070
[6,   900] loss:0.103
accuracy on test set:98 %
[7,   300] loss:0.032
[7,   600] loss:0.064
[7,   900] loss:0.092
accuracy on test set:98 %
[8,   300] loss:0.027
[8,   600] loss:0.053
[8,   900] loss:0.082
accuracy on test set:98 %
[9,   300] loss:0.025
[9,   600] loss:0.048
[9,   900] loss:0.073
accuracy on test set:99 %
[10,   300] loss:0.024
[10,   600] loss:0.045
[10,   900] loss:0.067
accuracy on test set:98 %
